### Set up functions and routes

In [1]:
def get_time(location: str) -> str:
    """Useful to get the time in a specific location"""
    print(f"Calling `get_time` function with location: {location}")
    return "get_time"


def get_news(category: str, country: str) -> str:
    """Useful to get the news in a specific country"""
    print(
        f"Calling `get_news` function with category: {category} and country: {country}"
    )
    return "get_news"

Now generate a dynamic routing config for each function

In [2]:
from semantic_router.route import Route, RouteConfig

functions = [get_time, get_news]
routes = []

for function in functions:
    route = await Route.from_dynamic_route(entity=function)
    routes.append(route)

route_config = RouteConfig(routes=routes)

/Users/jakit/customers/aurelio/semantic-router/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
2023-12-19 16:06:38 INFO semantic_router.utils.logger Generating dynamic route...
2023-12-19 16:06:44 INFO semantic_router.utils.logger Generated route config:
{
    "name": "get_time",
    "utterances": [
        "What's the time in [location]?",
        "Can you tell me the time in [location]?",
        "I need to know the time in [location].",
        "What time is it in [location]?",
        "Can you give me the time in [location]?"
    ]
}
2023-12-19 16:06:44 INFO semantic_router.utils.logger Generating dynamic route...
2023-12-19 16:06:50 INF

In [4]:
# You can manually add or remove routes

get_weather_route = Route(
    name="get_weather",
    utterances=[
        "what is the weather in SF",
        "what is the current temperature in London?",
        "tomorrow's weather in Paris?",
    ],
)
route_config.add(get_weather_route)

route_config.remove("get_weather")

route_config.to_dict()

2023-12-19 16:07:10 INFO semantic_router.utils.logger Added route `get_weather`
2023-12-19 16:07:10 INFO semantic_router.utils.logger Removed route `get_weather`


[{'name': 'get_time',
  'utterances': ["What's the time in [location]?",
   'Can you tell me the time in [location]?',
   'I need to know the time in [location].',
   'What time is it in [location]?',
   'Can you give me the time in [location]?'],
  'description': None},
 {'name': 'get_news',
  'utterances': ['Tell me the latest news from the US',
   "What's happening in India today?",
   'Get me the top stories from Japan',
   'Can you give me the breaking news from Brazil?',
   "What's the latest in Germany?"],
  'description': None}]

In [5]:
# Get a route by name
route_config.get("get_time")

Route(name='get_time', utterances=["What's the time in [location]?", 'Can you tell me the time in [location]?', 'I need to know the time in [location].', 'What time is it in [location]?', 'Can you give me the time in [location]?'], description=None)

Save config to a file (.json or .yaml)

In [5]:
route_config.to_file("route_config.json")

2023-12-19 16:04:24 INFO semantic_router.utils.logger Saving route config to route_config.json


Load from local file

In [6]:
route_config = RouteConfig.from_file("route_config.json")

2023-12-19 16:07:16 INFO semantic_router.utils.logger Loading route config from route_config.json


### Define routing layer

In [8]:
from semantic_router import RouteLayer

route_layer = RouteLayer(routes=route_config.routes)

Do a function call with functions as tool

In [9]:
from semantic_router.utils.function_call import route_and_execute

tools = [get_time, get_news]

await route_and_execute(
    query="What is the time in Stockholm?", functions=tools, route_layer=route_layer
)
await route_and_execute(
    query="What is the tech news in the Lithuania?",
    functions=tools,
    route_layer=route_layer,
)
await route_and_execute(query="Hi!", functions=tools, route_layer=route_layer)

2023-12-19 16:07:25 INFO semantic_router.utils.logger Extracting function input...


Calling function: get_time


2023-12-19 16:07:27 ERROR semantic_router.utils.logger Input name missing from query


Inputs: {'location': 'Stockholm'}
Schema: {'name': 'get_time', 'description': 'Useful to get the time in a specific location', 'signature': '(location: str) -> str', 'output': "<class 'str'>"}


ValueError: Invalid inputs